# Mini-Lab -- Pickle Models

This mini-lab is a follow-up to some in-class activities we've done.
It intentionally doesn't remind you explicitly how to do stuff, because
part of the activity is to learn how to do these things on your own!


## Hints

Read this: https://scikit-learn.org/stable/modules/model_persistence.html


## Deliverable

Complete the following functions, fulfilling the requirements listed in their docstrings:
* `serialize_model`
* `load_model` 
* `load_prediction_data`

Submit a completed version of this notebook, in `.ipynb` format.

# Do imports...

In [1]:
from pathlib import Path

import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import numpy as np

import pickle

np.random.seed(0)

In [2]:
def load_phish_storm():
    if Path('./PhishStorm-phishing-urls.zip').is_file():
        print(f'PhishStorm-phishing-urls.zip already exists in local')
    else:
        !wget https://storage.googleapis.com/security-analytics-datasets-public/PhishStorm-phishing-urls.zip
    
    if Path('./PhishStorm-phishing-urls.csv').is_file():
        print(f'PhishStorm-phishing-urls.csv already exists in local')
    else:
        !unzip PhishStorm-phishing-urls.zip

    df = pd.read_csv("PhishStorm-phishing-urls.csv", encoding_errors='ignore', on_bad_lines='skip', low_memory=False)
    return df

df = load_phish_storm()

--2022-03-01 06:44:35--  https://storage.googleapis.com/security-analytics-datasets-public/PhishStorm-phishing-urls.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.69.240, 142.250.72.16, 142.250.72.48, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.69.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3259236 (3.1M) [application/zip]
Saving to: ‘PhishStorm-phishing-urls.zip’

PhishStorm-phishing 100%[===================>]   3.11M  1.63MB/s    in 1.9s    

2022-03-01 06:44:37 (1.63 MB/s) - ‘PhishStorm-phishing-urls.zip’ saved [3259236/3259236]

Archive:  PhishStorm-phishing-urls.zip
  inflating: PhishStorm-phishing-urls.csv  


## Peek at the dataframe...

This is provided to show you what the data should look like once you have loaded it.

In [3]:
# https://stackoverflow.com/questions/48997644/how-to-describe-columns-as-categorical-values
df.astype('object').describe().transpose()

,count,unique,top,freq
domain,96002,96000,'www.allegropl.xaa.pl/enter_login.html?session...,2
ranking,95951,7016,10000000,56065
mld_res,95935,19,0.0,52217
mld.ps_res,95924,8,0.0,76496
card_rem,95923.0,53.0,2.0,17179.0
ratio_Rrem,95923.0,10042.0,0.0,5632.0
ratio_Arem,95923.0,10231.0,0.0,5686.0
jaccard_RR,95922.0,5446.0,0.0,76866.0
jaccard_RA,95921.0,5628.0,0.0,75286.0
jaccard_AR,95920.0,5071.0,0.0,77689.0


## Fit a model and save it to disk

In this section, we first simulate the machine learning process of
data preparation, model fitting, and model evaluation. 

At the end of this section, we serialize (save) our fitted model to disk using 
python's `pickle` library.

### Clean

In [4]:
df = df.dropna()

### Feature Engineer

We create a feature calculated from the length of the url.

In [5]:
df['domain_len'] = df['domain'].apply(lambda x: len(x))

### Partition the data

We select our new column to be the sole predictor, and we calculate a test-train split.
We fit a Logistic Regression model using model defaults.

In [6]:
X = df[['domain_len']]
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Save some data to disk, to use later in the deployment phase

We now save some testing data to disk in JSON format.
Later, we will load it back, and ask our unserialized model
to get predictions for it. This will simulate our serialized model
having been deployed somewhere, and having been asked to make
predictions against data sent in json format.

In [7]:
X.sample(n=5).to_json('test.json')

### Fit a model

In [8]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression()

### Evaluate the model

In [9]:
y_pred = clf.predict(X_test)
accuracy_score(y_pred, y_test)

0.752801960068811

Not a bad performance considering we only used one predictor...

### Save the model to disk

In this step, we save our fitted model to disk via serializing it.

In [10]:
# Serialize your fitted sklearn classifier (`clf`) to disk.
# hint: https://docs.python.org/3/library/pickle.html#examples
def serialize_model(clf):
    with open('df.pickle', 'wb') as f:
        pickle.dump(clf, f, pickle.HIGHEST_PROTOCOL) # Pickle the 'data' dictionary using the highest protocol available.
        
serialize_model(clf)

## Use the deployed the model

Imagine that the code in the following section is running in a separate context 
from all code that ran before it. 

**Important!** When I run your code, I should be able
to successfully run each of the cells below *even if none of the previous cells have been run within the current kernel*,
(assuming the pickled model file and json data exist on disk).

You can test your code the way I will by doing the following:

1. first running all cells so that the files are written to disk
1. then running "Restart Kernel" from the jupyter "Kernel" menu,
1. and then by manually running the cell below.

In [2]:
import pickle
import pandas as pd

#load your fitted sklearn classifier from disk. returns: sklearn classifier
#hint: https://docs.python.org/3/library/pickle.html#examples

def load_model():
    
    with open('df.pickle', 'rb') as f:
        clf_loaded = pickle.load(f)
    return clf_loaded

clf_loaded = load_model()

# load your json data into a pandas data frame. Earlier, we saved it to disk as filename `test.json`
# hint: https://pandas.pydata.org/docs/reference/api/pandas.read_json.html. returns: pandas df

def load_prediction_data():
    test_data_sample = pd.read_json("test.json")
    return test_data_sample

# Load the prediction data
test_data_sample = load_prediction_data()

# Get predictions for the uploaded data
clf_loaded.predict(test_data_sample)

array([1., 1., 0., 0., 0.])